# ***Credit Risk Stress Testing and Monte Carlo Simulation***

In [24]:
import numpy as np

In [ ]:
from google.colab import files
files.upload()

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle\ \(1\).json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!ls ~/.kaggle


!kaggle datasets list | head


kaggle.json
ref                                                            title                                               size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
saidaminsaidaxmadov/chocolate-sales                            Chocolate Sales                                   468320  2026-01-04 14:23:35.490000          14959        264  1.0              
aliiihussain/amazon-sales-dataset                              Amazon_Sales_Dataset                             1297759  2026-02-01 11:37:12.353000           2327         48  1.0              
hassanjameelahmed/store-sales                                  Retail Store Sales                                 66677  2026-02-04 06:31:47.217000            649         27  1.0              
wardabilal/customer-sho

In [5]:
!kaggle datasets download -d uciml/default-of-credit-card-clients-dataset


Dataset URL: https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset
License(s): CC0-1.0
  0% 0.00/0.98M [00:00<?, ?B/s]
100% 0.98M/0.98M [00:00<00:00, 1.01GB/s]


In [6]:
!unzip default-of-credit-card-clients-dataset.zip


Archive:  default-of-credit-card-clients-dataset.zip
  inflating: UCI_Credit_Card.csv     


In [14]:
import pandas as pd

df = pd.read_csv("UCI_Credit_Card.csv")
df.head()


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [15]:
df.columns = (df.columns
              .str.strip()
              .str.lower()
              .str.replace(' ', '_')
              .str.replace('.', '_'))

In [16]:
df.head(2)

,id,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,...,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default_payment_next_month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1


In [17]:
df = df.rename(columns={'default_payment_next_month':'default'})

In [18]:
df

,id,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,...,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000.0,1,3,1,39,0,0,0,0,...,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,29997,150000.0,1,3,2,43,-1,-1,-1,-1,...,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,29998,30000.0,1,2,2,37,4,3,2,-1,...,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,29999,80000.0,1,3,1,41,1,-1,0,0,...,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1




*   We are using Credit Risk Dataset to study average default rate for portfolio and how worse lossess can get (baseline)
*   Typical monthly defaults


*   Bad Month risk (95th percentile)
*   Extreme Tail risk (99th percentile)




*   Impact of Recession Scenarios on Portfolio Lossess abd financial stress on expected tail lossess












# Baseline Default Rate

*   Math : p_hat = sum(Default_j)/N



In [22]:
p_hat = df['default'].mean()
print(f"Overall Portfolio Default Rate: {p_hat:.2%}")

Overall Portfolio Default Rate: 22.12%


# Confidence Intervals


*   Math : SE(p_hat) = sqrt(1-p_hat)*p_hat/N
*   lower bound = p_hat - 1.96 * SE
*   upper bound = p_hat + 1.96 * SE

    








In [26]:
SE = np.sqrt((1-p_hat)*p_hat/len(df))
print(f"Standard Error: {SE:.2%}")

Standard Error: 0.24%


In [27]:
lower_bound = p_hat - 1.96 * SE
upper_bound = p_hat + 1.96 * SE

print(f"95% Confidence Interval: [{lower_bound:.2%}, {upper_bound:.2%}]")

95% Confidence Interval: [21.65%, 22.59%]


# Monte Carlo Simulation (Normal Economy)


*   Math : D(i) ~ Binomial(n,p) for i = 1... M




In [32]:
monthly_defaults = np.random.binomial(n=10_000, p=p_hat, size=500)
print(f"Monthly Defaults: {monthly_defaults}")


Monthly Defaults: [2172 2215 2272 2208 2219 2214 2189 2267 2173 2143 2302 2131 2162 2195
 2259 2230 2242 2177 2171 2159 2283 2171 2143 2203 2179 2183 2176 2219
 2148 2209 2202 2173 2216 2210 2201 2224 2194 2190 2317 2195 2240 2192
 2264 2239 2242 2221 2246 2209 2190 2271 2264 2273 2205 2169 2204 2201
 2300 2189 2233 2195 2147 2203 2106 2150 2251 2241 2185 2236 2144 2177
 2215 2210 2110 2216 2225 2224 2237 2205 2196 2220 2225 2133 2164 2218
 2247 2260 2226 2206 2298 2240 2176 2257 2246 2301 2189 2207 2173 2213
 2188 2211 2178 2175 2179 2269 2108 2158 2215 2206 2204 2269 2196 2108
 2207 2221 2222 2214 2255 2186 2245 2186 2233 2191 2250 2198 2167 2187
 2217 2169 2237 2213 2221 2241 2209 2168 2193 2209 2251 2255 2198 2202
 2195 2264 2302 2134 2236 2209 2159 2259 2212 2216 2240 2263 2236 2292
 2152 2242 2206 2219 2223 2158 2119 2277 2187 2215 2230 2176 2213 2192
 2274 2218 2307 2211 2217 2190 2205 2154 2209 2228 2202 2276 2180 2216
 2210 2327 2229 2215 2243 2190 2259 2269 2241 2300 2178 218

# Tail Risk Estimation Quantile_q({D_(i)})

In [33]:
np.percentile(monthly_defaults, [50, 95, 99])

array([2211.5 , 2280.  , 2302.05])

# Stress Testing (Recession Scenario)



*   Math : p_stress = 0.30
*   D_stress(i) ~ Binomial(n, p_stress)



In [34]:
p_stress = 0.30
stress_defaults = np.random.binomial(n=10_000, p=p_stress, size=500)
print(f"Monthly Defaults: {monthly_defaults}")

Monthly Defaults: [2172 2215 2272 2208 2219 2214 2189 2267 2173 2143 2302 2131 2162 2195
 2259 2230 2242 2177 2171 2159 2283 2171 2143 2203 2179 2183 2176 2219
 2148 2209 2202 2173 2216 2210 2201 2224 2194 2190 2317 2195 2240 2192
 2264 2239 2242 2221 2246 2209 2190 2271 2264 2273 2205 2169 2204 2201
 2300 2189 2233 2195 2147 2203 2106 2150 2251 2241 2185 2236 2144 2177
 2215 2210 2110 2216 2225 2224 2237 2205 2196 2220 2225 2133 2164 2218
 2247 2260 2226 2206 2298 2240 2176 2257 2246 2301 2189 2207 2173 2213
 2188 2211 2178 2175 2179 2269 2108 2158 2215 2206 2204 2269 2196 2108
 2207 2221 2222 2214 2255 2186 2245 2186 2233 2191 2250 2198 2167 2187
 2217 2169 2237 2213 2221 2241 2209 2168 2193 2209 2251 2255 2198 2202
 2195 2264 2302 2134 2236 2209 2159 2259 2212 2216 2240 2263 2236 2292
 2152 2242 2206 2219 2223 2158 2119 2277 2187 2215 2230 2176 2213 2192
 2274 2218 2307 2211 2217 2190 2205 2154 2209 2228 2202 2276 2180 2216
 2210 2327 2229 2215 2243 2190 2259 2269 2241 2300 2178 218

# Financial Loss Modeling



*   Math : Loss_per_default (assumption) = 50_000
*   Monthly_loss = monthly_default * loss_per_default
*   E[loss] = loss * E[D]



In [46]:
loss_per_default = 50_000
monthly_loss = monthly_defaults * loss_per_default
stress_loss = stress_defaults * loss_per_default

np.percentile(monthly_loss, [50, 95, 99])


array([1.105750e+08, 1.140000e+08, 1.151025e+08])

In [55]:
print(f"Normal Loss under Economic conditions : {np.mean(monthly_loss)}")
print(f"Stress Loss : {np.mean(stress_loss)}")
print(f"Incremental Loss : ", stress_loss.mean() - monthly_loss.mean())


Normal Loss under Economic conditions : 110624500.0
Stress Loss : 150165800.0
Incremental Loss :  39541300.0


Under recession scenario, expected monthly loss increase from ~ 1.1 crore to 1.5 crore for a 10,000 customer portfolio, representing an incremental impact of about ~40 lakhs